**Playlist Title Prediction with NMT using Encoder-Decoder LSTM model** by Chelsea Vickers and Mallory Price

In [9]:
import pandas as pd
import numpy as np
import os, string
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers import LSTM, Embedding, Input, Dense
from keras.models import Model

In [2]:
#read data from csv into pandas df
data = pd.read_csv("../Neural-Nets-Final-Project/spotify.csv", encoding = "ISO-8859-1")
data.head()

,Playlist Title,Track List
0,newwwww,"['Linger', 'Best Friend', 'Wash It Away', 'Fre..."
1,junior year,"['Dreams - 2004 Remaster', 'Go Your Own Way - ..."
2,running away,"['Closing Time', 'Call Me Maybe', ""Honey, I'm ..."
3,jam,"['Go Your Own Way - 2004 Remaster', 'You Reall..."
4,cc/summer'18,"['Waves', 'Needles & Pins', 'Firework', 'Ring'..."


In [3]:
#preprocessing

#remove punctuation from titles and tracks
exclude = set(string.punctuation)
data['Playlist Title'] = data['Playlist Title'].apply(lambda x : ''.join(ch for ch in x if ch not in exclude))
data['Track List'] = data['Track List'].apply(lambda x : ''.join(ch for ch in x if ch not in exclude))

#make all characters lowercase
data['Playlist Title'] = data['Playlist Title'].apply(lambda x : x.lower())
data['Track List'] = data['Track List'].apply(lambda x : x.lower())

#add start and end tokens to the target titles
data['Playlist Title'] = data['Playlist Title'].apply(lambda x : 'START_ ' + x + ' _END')

#get lists of the titles and tracks from the pandas df
titles = data['Playlist Title'].to_list()
tracks = data['Track List'].to_list()

In [4]:
#get title and track vocabularies

title_words = set()
for title in titles:
    for word in title.split():
        title_words.add(word)

track_words = set()
for track in tracks:
    for word in track.split():
        track_words.add(word)

In [5]:
# Max Length of track list
max_len_tracks = 0
for l in data['Track List']:
    if len(l.split(' ')) > max_len_tracks:
        max_len_tracks = len(l.split(' '))

# Max Length of title
max_len_title = 0
for l in data['Playlist Title']:
    if len(l.split(' ')) > max_len_title:
        max_len_title = len(l.split(' '))

In [6]:
input_words = sorted(list(track_words))
target_words = sorted(list(title_words))
num_encoder_tokens = len(track_words)
num_decoder_tokens = len(title_words)+1 # +1 for zero-padding

In [7]:
#dictionaries for converting tokens into their corresponding indices (and reverse)
#for both title and track vocabularies

input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [11]:
data = shuffle(data)
data.head()

,Playlist Title,Track List
7968,START_ winter 1819 _END,sunflower spiderman into the spiderverse ever...
5271,START_ preshow pregame _END,hey mami nasa break up with your girlfriend im...
3145,START_ new years eve by icona pop _END,slippery slope jonny boy sound of violence aer...
2272,START_ 1968 _END,love is blue honey 1991 remaster sittin on t...
8285,START_ summer camp feel good _END,el perdãâãâãâãâ³n hey ya yaalili wavin ...


In [12]:
#separate the data into training (90%) and testing data (10%)

x,y = data['Track List'], data['Playlist Title']
x_train,x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
x_train.shape, x_test.shape

((9021,), (1003,))

In [13]:
#generator function which generates a batch of data, given the training data and a specified batch size

def gen_batch(x=x_train, y=y_train, batch_size=128):
    while True:
        for i in range(0,len(x),batch_size):
            encoder_input_data = np.zeros((batch_size, max_len_tracks), dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_len_title), dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_len_title, num_decoder_tokens), dtype='float32')
            for j,(input_text, target_text) in enumerate(zip(x[i:i+batch_size], y[i:i+batch_size])):
                for t,word in enumerate(input_text.split()):
                    encoder_input_data[j,t]=input_token_index[word]
                for t,word in enumerate(target_text.split()):
                    if t < len(target_text.split())-1:
                        decoder_input_data[j,t]=target_token_index[word]
                    if t > 0:
                        decoder_target_data[j, t-1, target_token_index[word]]=1
            yield([encoder_input_data, decoder_input_data],decoder_target_data)

In [14]:
#build the encoder-decoder model

latent_dim = 50

#encoder

encoder_inputs = Input(shape=(None,))
enc_embed = Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_embed)
encoder_states = [state_h, state_c]


#decoder

decoder_inputs = Input(shape=(None,))
dec_embed_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_embed = dec_embed_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_embed, initial_state=encoder_states)
decoder_dense_layer = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense_layer(decoder_outputs)


#encoder-decoder model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [15]:
#compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [16]:
#specify the training parameters and configure the model for training

train_samples = len(x_train)
val_samples = len(x_test)
batch_size = 128
epochs = 20

model.fit_generator(generator = gen_batch(x_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = gen_batch(x_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/Users/chelseavickers/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
70/70 [==============================] - 257s 4s/step - loss: 1.1856 - acc: 0.2717 - val_loss: 0.9974 - val_acc: 0.2800
Epoch 2/20
70/70 [==============================] - 249s 4s/step - loss: 0.9844 - acc: 0.2763 - val_loss: 1.0249 - val_acc: 0.2768
Epoch 3/20
70/70 [==============================] - 246s 4s/step - loss: 0.9667 - acc: 0.2769 - val_loss: 1.0255 - val_acc: 0.2792
Epoch 4/20
70/70 [==============================] - 247s 4s/step - loss: 0.9491 - acc: 0.2769 - val_loss: 0.9130 - val_acc: 0.2790
Epoch 5/20
70/70 [==============================] - 247s 4s/step - loss: 0.9355 - acc: 0.2774 - val_loss: 1.0809 - val_acc: 0.2797
Epoch 6/20
70/70 [==============================] - 246s 4s/step - loss: 0.9241 - acc: 0.2820 - val_loss: 0.9813 - val_acc: 0.2870
Epoch 7/20
70/70 [==============================] - 247s 4s/step - loss: 0.9130 - acc: 0.2833 - val_loss: 1.1138 - val_acc: 0.2842
Epoch 8/20
70/70 [==============================] - 245s 3s/step - loss: 0.9049 - a

In [17]:
#save the weights
model.save_weights('nmt_weights.h5')

In [ ]:
#if needed, load the weights
model.load_weights('nmt_weights.h5')

In [18]:
#encode the input sequence
encoder_model = Model(encoder_inputs, encoder_states)

decoder_states_inputs = [Input(shape=(latent_dim,)),Input(shape=(latent_dim,))]

#get the decoder sequence embeddings
dec_embed2=dec_embed_layer(decoder_inputs)

#set the initial states to the states from previous time step to predict next word in the sequence
decoder_outputs2,state_h2,state_c2 = decoder_lstm(dec_embed2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2,state_c2]
decoder_outputs2 = decoder_dense_layer(decoder_outputs2)

#final decoder
decoder_model = Model([decoder_inputs]+decoder_states_inputs,[decoder_outputs2]+decoder_states2)

In [19]:
#function to decode input sequences

def decode_seq(input_seq):
    #encode input as state vectors
    states_value = encoder_model.predict(input_seq)
    
    #initialize empty target sequence of size 1
    target_seq = np.zeros((1,1))
    target_seq[0,0] = target_token_index['START_']
    
    #loop through batch of sequences (assume batch size = 1 for simplification purposes)
    stop = False
    decoded = ''
    while not stop:
        output_tokens,h,c = decoder_model.predict([target_seq]+states_value)
        
        #get a sample output token
        sample_token_index = np.argmax(output_tokens[0,-1,:])
        sample_char = reverse_target_char_index[sample_token_index]
        decoded += ' ' + sample_char
        
        #trigger exit condition because either at the end of the sample token or hit max length
        if (sample_char=='_END' or len(decoded)>50):
            stop = True
        
        #update target sequence and states
        target_seq = np.zeros((1,1))
        target_seq[0,0] = sample_token_index
        states_value=[h,c]
        
    return decoded

In [20]:

#evaluate with training data


In [22]:
train_gen = gen_batch(x_train,y_train,batch_size=1)
k=-1

In [23]:
k+=1
(input_seq,actual_output),_=next(train_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_train[k:k+1].values[0])
print('Actual Playlist Title:', y_train[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501. 5763. 3278.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: monica feat tech n9ne 100 bands whoop tang flow feat raekwon minor gangsters gully i was sad last night im ok now top of the morning what we on diet ego check dangerookipawaa freestyle jumpin in rome ginobili cameras eggs industry games dmtri metal wings iceman voices slip draino feat denzel curry hunnid feat joyce wrice bye felicia wassup phonies angelic hoodrat the bean kicked in handful hollyhood feat kent jamz shrimp at phillipes feat benny the butcher moon  stars feat maggie lindemann on me treehouse what i please feat denzel curry george bondo all dogs go to heaven meathead carbonated water first to last pisces distance beautiful brixton miami fuck your culture dr birds i heard you were looking for me ryu  ken drug lullaby different break it down magical
Actual Playlist Title:  spilled ink 
Predicted Playlist Title:  cold 


In [24]:
k+=1
(input_seq,actual_output),_=next(train_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_train[k:k+1].values[0])
print('Actual Playlist Title:', y_train[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501. 1553. 3608.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: ftarri  tokyo  oct 14th 2013 simulacrum ivii refound  1 inference to the best explanation symbiogenesis osa vaya for organ and brass breath for organ
Actual Playlist Title:  claire krtu 
Predicted Playlist Title:  editors de ãâãâ¢ ãâãâ¢ ãâãâ¢ ãâãâ¢ ãâ


In [25]:
k+=1
(input_seq,actual_output),_=next(train_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_train[k:k+1].values[0])
print('Actual Playlist Title:', y_train[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501. 6126. 3799.  688. 6724. 5180.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: please please me  mono  remastered twist and shout  remastered i saw her standing there  remastered baby its you  remastered she loves you  mono  remastered i want to hold your hand  remastered 2015 roll over beethoven  remastered i wanna be your man  remastered till there was you  remastered money thats what i want  remastered cant buy me love  remastered 2015 and i love her  remastered a hard days night  remastered eight days a week  remastered rock and roll music  remastered everybodys trying to be my baby  remastered ticket to ride  remastered youve got to hide your love away  remastered yesterday  remastered nowhere man  remastered im looking through you  remastered norwegian wood this bird has flown  remastered michelle  remastered drive my car  remastered got to get you into my life  remastered im only sleeping  remastered and your bir

In [26]:
k+=1
(input_seq,actual_output),_=next(train_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_train[k:k+1].values[0])
print('Actual Playlist Title:', y_train[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501. 6076.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: swimming in the moonlight perfect places toothpaste kisses winter of our youth open season give it time silver moon moon river from breakfast at tiffanys remastered mad sounds buzzcut season wait for me the loved ones sleep alone warmth dancing on quicksand cest la vie dancing in the dark gemini feed angela sweet disposition rock bottom first love are you calling africa ill keep you safe after the storm in a week feat karen cowley 17 in a little while up  theme from the motion picture the moon song  studio version duet black mambo hallelujah come on eileen space song house of gold roman holiday somewhere only we know tongue tied oblivion audition the fools who dream mia  sebastianãâãâ¢ãâãâãâãâs theme drops of jupiter the girl cherry wine  live sleep on the floor flaws the rain the park  other things  single version supercut the lou

In [27]:
k+=1
(input_seq,actual_output),_=next(train_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_train[k:k+1].values[0])
print('Actual Playlist Title:', y_train[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501. 2568. 3662.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: in  out  baardsen remix cadillac girl i am the lion king molasses fiona coyne wont get away omen  radio edit call you out lines  demo version nakamarra cant you tell reconstruct feat seafloor the less i know the better lost tonight real june past lives i know theres gonna be good times feat popcaan watch my back nevermind the end distance my girls them changes feat flying lotus  kamasi washington afro blue loversãâãâ¢ãâãâãâãâ carvings country a calf born in winter 20th century fix  warm weather since i left you decisions pressure realla falling short why didnt you call mothers love dont stretch might be girl feat kaytranada stay with us broken soldier
Actual Playlist Title:  for laura 
Predicted Playlist Title:  summer 


In [28]:
k+=1
(input_seq,actual_output),_=next(train_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_train[k:k+1].values[0])
print('Actual Playlist Title:', y_train[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501. 6002. 5685.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: sol vind och vatten sommartider sommaren ãâãâãâãâ¤r kort sommaren i city ãâãâãâãâppna landskap dina fãâãâãâãâ¤rger var blãâãâãâãâ¥ ãâãâãâãânglamark att angãâãâãâãâ¶ra en brygga oh vilken hãâãâãâãâ¤rlig da om ett sista glas kung av sand visa vid vindens ãâãâãâãâ¤ngar sommarnatt  1999 hon kommer med solsken en dag pãâãâãâãâ¥ stranden vãâãâãâãâ¥gorna nãâãâãâãâ¤r vi grãâãâãâãâ¤ver guld i usa en kvãâãâãâãâ¤ll i juni juni juli augusti oh boy omãâãâãâãâ¥omigen ta mig till havet dansa pausa de dagar vi drãâãâãâãâ¶mt om vingar himlen ãâãâãâãâ¤r oskyldigt blãâãâãâãâ¥ smãâãâãâãâ¥ lãâãâãâãâ¤tta moln  studioversion sommar och sol sommarkort ingen sommar utan reggae gãâãâãâãâ¶r mig lycklig nu magaluf hãâãâãâãâ

In [47]:
k+=1
(input_seq,actual_output),_=next(train_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_train[k:k+1].values[0])
print('Actual Playlist Title:', y_train[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501. 2023. 2104.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: another you paradise voices lean on say youll be there holding on f for you you know you like it lost  found  kastle remix bang that need u 100  radio edit i got u beggin for thread warm water  snakehips remix beautiful now find you straight into the fire addicted to a memory under control feat hurts love now feat all about she dollar signs feat tinashe for you this is what it feels like underneath the sky do or die  afrojack vs thirty seconds to mars remix born to run firefly  bravado dubstep remix bad girls heads will roll  atrak remix i cant stop drinking about you  chainsmokers remix blind heart  radio edit up we go running with the boys i cant stop drinking about you replay hide away renegades where are ãâãâãâãâ now with justin bieber kanye cool for the summer high for this the hills
Actual Playlist Title:  dj driving 
Predicted 

In [29]:

#evaluate with testing data


In [30]:
test_gen = gen_batch(x_test, y_test, batch_size = 1)
k=-1

In [31]:
k+=1
(input_seq,actual_output),_=next(test_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_test[k:k+1].values[0])
print('Actual Playlist Title:', y_test[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501. 5711. 5386.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: daylight feat daniela andrade ryan hemsworth lontalius  olli bara du every time i go home find wild feat khai its you forever being arms out of focus friends  recorded at spotify studios nyc wake up full moon song coffee amadeus all the time dont worry be happy where you are  acoustic electric feel arc drift here if you want pale blue better better off home flowers in your hair boardwalks  sonny alven remix higher ground feat naomi wild gold angel truth apocalypse catch  release deepend remix extended version direction pulling me under older sensitive heart ayahuasca
Actual Playlist Title:  soul search 
Predicted Playlist Title:  starred 


In [32]:
k+=1
(input_seq,actual_output),_=next(test_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_test[k:k+1].values[0])
print('Actual Playlist Title:', y_test[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501. 6126. 6636. 6899. 5715.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: amsterdam rounder holocene asleep  2008 remaster the stable song real love baby pretty girl from cedar lane the muse jackie and wilson you are the best thing 40 day dream iplayyoulisten amsterdam peaches ghosts sedona caring is creepy little lies  2017 remaster my kind of lady tompkins square park dreams  2004 remaster slow motion rhiannon coming home tiger striped sky bloodflood dream a little dream of me blue ruin here in my arms light year my eyes unforgettable op jebediah majorette she always takes it black wallowa lake monster verona home pt 1  2 the ballad of keenan milton patience the universe
Actual Playlist Title:  the way you sound 
Predicted Playlist Title:  spring jams 


In [33]:
k+=1
(input_seq,actual_output),_=next(test_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_test[k:k+1].values[0])
print('Actual Playlist Title:', y_test[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501. 5687.  895.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: sexual what lovers do feat sza how long how long locked out of heaven roxanne  remastered 2003 im not your toy rio graceland unbelievers girl almighty 5 oclock feat lily allen  wiz khalifa whod have known shine white ferrari here there and everywhere  remastered 2009 god only knows  remastered get free creep the air that i breathe  2008 remaster mr watson sloop john b  remastered call my dad torn the lengths sunny runaway sinner boots not fair rainbowland feat dolly parton two ghosts you and i man i feel like a woman the wire nothing can change this love lovin touchin squeezin 40 day dream since were alone dreams  2004 remaster childs play feat chance the rapper blessings feat ty dolla ign anderson paak bj the chicago kid raury  jamila woods in your phone with lauren jauregui strangers the feeling deja vu homemade dynamite  remix location fea

In [34]:
k+=1
(input_seq,actual_output),_=next(test_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_test[k:k+1].values[0])
print('Actual Playlist Title:', y_test[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501.  814. 5717.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: dearly departed feat esmãâãâãâãâ© patterson piece of my heart the way we get by your hand in mine two matchsticks pride and joy street joy bad vibrations shes about a mover gods dream pepper bright lights are you alright bitch i love you good intentions lost coastlines summertime punk as fuck is anybody going to san antone  single version do you silverado bench seat on the road again  live naked as we came build me up from bones youre gonna miss me choctaw bingo back and forth pancho  lefty spring forward tuff enuff the ghost within lights  from sons of anarchy backwater i lie when i drink masollan man of the world a chronicle of early failures  part 1 cliffs of dover  instrumental there is a time feat kelsey wilson gold in the air dungtitled in a major
Actual Playlist Title:  austin sounds 
Predicted Playlist Title:  spring jams 


In [35]:
k+=1
(input_seq,actual_output),_=next(test_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_test[k:k+1].values[0])
print('Actual Playlist Title:', y_test[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501. 4492. 1285. 2800.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: the sweet escape yeah feat lil jon  ludacris
Actual Playlist Title:  oldies but goodies 
Predicted Playlist Title:  arab 


In [36]:
k+=1
(input_seq,actual_output),_=next(test_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_test[k:k+1].values[0])
print('Actual Playlist Title:', y_test[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501. 5232. 5173.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: san luis i forget myself feat darlingside our remains  acoustic rome good old days turntable samara beige ill be out here pharmakon by and by twist and its still alright ill be gone you would have to lose your mind strangers strangers the parting glass  live from the late late show backbone a trick of the light october the swimmer to be an old man you sigh dynamite people need a melody people change front porch such a simple thing real you wont be long bare cavalry the barrel silver lining better and better i wont run from it no woman glass fool i should live in salt  the national cover  live from spotify london i leave early eskimo coat side effects power over me  acoustic letter to madeline younger years justine like an eagle sedona johnny still life feat joseph hold your head up high love is everywhere beware punta cometa  dancing for the 

In [56]:
k+=1
(input_seq,actual_output),_=next(test_gen)
print(actual_output)
decoded=decode_seq(input_seq)
print('Tracks:', x_test[k:k+1].values[0])
print('Actual Playlist Title:', y_test[k:k+1].values[0][6:-4])
print('Predicted Playlist Title:', decoded[:-4])

[[ 501. 2155. 6696.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
Tracks: strawberry smoothie say it aint so told you id be with the guys where is my mind lithium tonight im wearing silk california here we go softer basket case androgynous mind i was born a unicorn kids hello operator i wanna be adored  remastered
Actual Playlist Title:  strawberry smooooothue 
Predicted Playlist Title:  cold jams 
